In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/worachotn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/worachotn/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/worachotn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/worachotn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from nltk.corpus import wordnet
from nltk.corpus import stopwords

lemmatizer = nltk.stem.WordNetLemmatizer()  # Initiate nltk lemmatizer

In [3]:
import math
import os
import pprint
import logging

import datasets
import nltk
import numpy as np
import torch
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from filelock import FileLock
from transformers import AdamW, get_scheduler, set_seed

from transformers.file_utils import is_offline_mode
from transformers.utils.versions import require_version

from args import parse_args
from data_loader import raw_data_loader, data_processor
from model_loader import model_loader
from rouge_s import py_rouge_scores
from utils import label_smoothed_nll_loss, postprocess_text
from new_loss import margin_ranking_loss, cosine_embedding_loss

[nltk_data] Downloading package punkt to /home/worachotn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/worachotn/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/worachotn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/worachotn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [5]:
from transformers import (
    MODEL_MAPPING,
    SchedulerType,
)

# You should update this to your particular problem to have better documentation of `model_type`
MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [6]:
import argparse
arg_parser = argparse.ArgumentParser(description="BART")
arg_parser.add_argument("--len_input", dest="len_input", type=str, default=None, help="set up prefix input",choices=('no', 'topic', 'length', 'topic-length', 'length-topic', 'simple', 'simple-topic-tagger', 'simple-tagger'))
arg_parser.add_argument("--len_output", dest="len_output", default=None, help="Use the ctrlen model or not", choices=('no', 'topic', 'length', 'topic-length', 'length-topic'))
arg_parser.add_argument("--output_dir", dest="output_dir", type=str, default="./output/1", help="default")
arg_parser.add_argument("--train_file", dest="train_file", type=str, default=None, help="A csv or a json file containing the training data.")
arg_parser.add_argument("--validation_file", dest="validation_file", type=str, default=None, help="A csv or a json file containing the validation data.")
arg_parser.add_argument("--test_file", dest="test_file", type=str, default=None, help="A csv or a json file containing the test data.")
arg_parser.add_argument("--ignore_pad_token_for_loss", dest="ignore_pad_token_for_loss", type=bool, default=True, help="Whether to ignore the tokens corresponding to " "padded labels in the loss computation or not.",)
arg_parser.add_argument("--text_column", dest="text_column", type=str, default="dialogue", help="The name of the column in the datasets containing the full texts (for summarization).")
arg_parser.add_argument("--summary_column", dest="summary_column", type=str, default="summary", help="The name of the column in the datasets containing the summaries (for summarization).")
arg_parser.add_argument("--model_name_or_path", dest="model_name_or_path", type=str, default="facebook/bart-large", help="Path to pretrained model or model identifier from huggingface.co/models.")
arg_parser.add_argument("--model_type", dest="model_type", type=str, default="bart", help="Model type to use if training from scratch.", choices=MODEL_TYPES)
arg_parser.add_argument("--max_source_length", dest="max_source_length", type=int, default=1024, help="default")
arg_parser.add_argument("--source_prefix", dest="source_prefix", type=str, default=None, help="A prefix to add before every source text " "(useful for T5 models).")
arg_parser.add_argument("--preprocessing_num_workers", type=int, default=None, help="The number of processes to use for the preprocessing.")
# arg_parser.add_argument("--overwrite_cache", dest="overwrite_cache", type=lambda x:bool(strtobool(x)), default=True, help="default")
arg_parser.add_argument("--overwrite_cache", dest="overwrite_cache", type=bool, default=None, help="Overwrite the cached training and evaluation sets")
arg_parser.add_argument("--min_target_length", dest="min_target_length", type=int, default=1, help="The minimal total sequence length for target text")
arg_parser.add_argument("--max_target_length", dest="max_target_length", type=int, default=128, help="The maximum total sequence length for target text after "
        "tokenization. Sequences longer than this will be truncated, sequences shorter will be padded."
        "during ``evaluate`` and ``predict``.")
arg_parser.add_argument("--num_beams", dest="num_beams", type=int, default=4, help="Number of beams to use for evaluation. This argument will be "
        "passed to ``model.generate``, which is used during ``evaluate`` and ``predict``.")
arg_parser.add_argument("--learning_rate", dest="learning_rate", type=float, default=5e-5, help="Initial learning rate (after the potential warmup period) to use.")
arg_parser.add_argument("--pad_to_max_length", action="store_true", help="If passed, pad all samples to `max_length`. Otherwise, dynamic padding is used.",)
arg_parser.add_argument("--weight_decay", dest="weight_decay", type=float, default=1e-3, help="Weight decay to use.")
arg_parser.add_argument("--label_smoothing", dest="label_smoothing", type=float, default=0.1, help="hyperparameter for label smoothing.")
arg_parser.add_argument("--length_penalty", dest="length_penalty", type=float, default=1.0, help="large - longer sequence, small - shorter sequence")
arg_parser.add_argument("--num_train_epochs", dest="num_train_epochs", type=int, default=15, help="Total number of training epochs to perform.")
arg_parser.add_argument("--per_device_train_batch_size", dest="per_device_train_batch_size", type=int, default=8, help="Batch size (per device) for the training dataloader.")
arg_parser.add_argument("--gradient_accumulation_steps", dest="gradient_accumulation_steps", type=int, default=64, help="Number of updates steps to accumulate before performing a backward/update pass.")
arg_parser.add_argument("--per_device_eval_batch_size", dest="per_device_eval_batch_size", type=int, default=8, help="Batch size (per device) for the evaluation dataloader.")
arg_parser.add_argument("--per_device_test_batch_size", dest="per_device_test_batch_size", type=int, default=8, help="Batch size (per device) for the evaluation dataloader.")
arg_parser.add_argument("--num_warmup_steps", dest="num_warmup_steps", type=int, default=0, help="Number of steps for the warmup in the lr scheduler.")
arg_parser.add_argument("--cache_dir", dest="cache_dir", type=str, default="./output/cache", help="default")
arg_parser.add_argument("--seed", dest="seed", type=int, default=12345, help="default")
# arg_parser.add_argument("-f", required=False) #important
arg_parser.add_argument("--config_name", type=str, default=None, help="Pretrained config name or path if not the same as model_name")
arg_parser.add_argument("--tokenizer_name", type=str, default=None, help="Pretrained tokenizer name or path if not the same as model_name")
arg_parser.add_argument("--use_slow_tokenizer", dest="use_slow_tokenizer", action="store_true", help="If passed, will use a slow tokenizer (not backed by the 🤗 Tokenizers library).")
arg_parser.add_argument("--max_train_steps", type=int, default=None, help="Total number of training steps to perform. If provided, overrides num_train_epochs.")
arg_parser.add_argument("--lr_scheduler_type", type=SchedulerType, default="linear", help="The scheduler type to use.", choices=["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"])
arg_parser.add_argument("--ctrlen_model", action='store_true', default=False, help="Use the ctrlen model or not")
arg_parser.add_argument("--sim_window_size", type=int, default=5, help="window size for computing loss.")
arg_parser.add_argument("--sim_loss", type=float, default=0, help="the loss weight for similarity scores.")
arg_parser.add_argument("--special_len_token_init", type=str, default=None, help="ways to initialize special token for length (random, zero, token_embs)")
arg_parser.add_argument("--embedding_lr", type=float, default=5e-5, help="Initial learning rate for embedding layers.")
arg_parser.add_argument("--len_start", type=int, default=1, help="start length.")
arg_parser.add_argument("--len_end", type=int, default=100, help="end length.")
arg_parser.add_argument("--data_aug",action='store_true',default=False,help="whether to perform data augmentation or not")
arg_parser.add_argument("--pred_len", action='store_true', default=False, help="whether to use the golden length or predicted length")
arg_parser.add_argument("--shuffle", action='store_true', default=False, help="whether to shuffle the dataset to balance train/validation/test")
arg_parser.add_argument("--debug", action='store_true', default=False, help="Use the debug mode or not")

arg_parser.add_argument("--topic_tagger", dest="topic_tagger", type=bool, default=None, help="Use topic tag <TAG> or not")
arg_parser.add_argument("--decoder_topic_tagger", dest="decoder_topic_tagger", type=bool, default=False, help="Use decoder topic tag [TAG] or not")
arg_parser.add_argument("--contrastive_loss", dest="contrastive_loss", type=bool, default=False, help="Use contrastive loss or not")
arg_parser.add_argument("--alpha", dest="alpha", type=float, default=0.5, help="ration of computation loss in encoder_hidden_layer")

_StoreAction(option_strings=['--alpha'], dest='alpha', nargs=None, const=None, default=0.5, type=<class 'float'>, choices=None, required=False, help='ration of computation loss in encoder_hidden_layer', metavar=None)

In [7]:
args = arg_parser.parse_args('')

In [8]:
args.train_file = "./data/dialogtest/dialogsum.train.jsonl"
args.validation_file = "./data/dialogtest/dialogsum.dev.jsonl"
args.test_file = "./data/dialogtest/dialogsum.test.jsonl"
args.text_column = "dialogue"
args.summary_column = "summary"
args.model_name_or_path = "facebook/bart-large"
args.model_type = "bart"
args.max_source_length = 1024
args.min_target_length = 1
args.max_target_length = 128
args.num_beams = 4
args.learning_rate = 5e-5
args.weight_decay = 1e-3
args.label_smoothing = 0.1
args.length_penalty = 1.0
args.num_train_epochs = 2
args.per_device_train_batch_size = 2
args.gradient_accumulation_steps = 64
args.per_device_eval_batch_size = 8
args.per_device_test_batch_size = 8
args.num_warmup_steps = 0
args.cache_dir = "./output/cache"
args.overwrite_cache = True
args.seed = 12345

args.len_input = 'topic-length'
args.len_output = 'topic'
args.output_dir = "./output/1-bart-baseline-loss"

args.topic_tagger = False
args.decoder_topic_tagger = False
args.contrastive_loss = True

In [9]:
print(args.len_input)
print(args.topic_tagger)
print(args.decoder_topic_tagger)
print(args.contrastive_loss)

topic-length
False
False
True


In [10]:
raw_datasets = raw_data_loader(args)

In [11]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'negative_dialogue', 'summary', 'negative_summary', 'topic', 'negative_topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'negative_dialogue', 'summary', 'negative_summary', 'topic', 'negative_topic'],
        num_rows: 50
    })
    test: Dataset({
        features: ['id', 'dialogue', 'negative_dialogue', 'summary', 'negative_summary', 'topic', 'negative_topic'],
        num_rows: 150
    })
})

In [12]:
raw_datasets['train']['dialogue'][0]

"Topic of Summary: get a check-up. Length of Summary: 30. Dialogue: #Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, than

In [13]:
raw_datasets['train']['negative_dialogue'][0]

"Topic of Summary: Valentine's Day. Length of Summary: 30. Dialogue: #Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, tha

In [14]:
raw_datasets['train']['summary'][0]

"Topic of Summary: get a check-up. Summary: Mr. Smith's getting a check-up, and Doctor Hawkins advises him to have one every year. Hawkins'll give some information about their classes and medications to help Mr. Smith quit smoking."

In [15]:
accelerator = Accelerator(mixed_precision="fp16")
logger.info(accelerator.state)

10/28/2023 15:55:52 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16



In [16]:
logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
accelerator.is_local_main_process
datasets.utils.logging.set_verbosity_warning()
transformers.utils.logging.set_verbosity_info()
set_seed(args.seed)
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
accelerator.is_main_process
os.makedirs(args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()

In [17]:
config, tokenizer, model = model_loader(accelerator, logger, args)

loading configuration file config.json from cache at ./output/cache/models--facebook--bart-large/snapshots/cb48c1365bd826bd521f650dc2e0940aee54720c/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2

In [18]:
model.vocab_size

50265

In [19]:
tokenizer.SPECIAL_TOKENS_ATTRIBUTES

['bos_token',
 'eos_token',
 'unk_token',
 'sep_token',
 'pad_token',
 'cls_token',
 'mask_token',
 'additional_special_tokens']

In [20]:
print(tokenizer.additional_special_tokens)

[]


In [21]:
dataloader, processed_dataset = data_processor(logger, args, accelerator, raw_datasets, tokenizer, model)
train_dataloader, eval_dataloader, test_dataloader = dataloader
train_dataset, _, _ = processed_dataset

Running tokenizer on dataset:   0%|          | 0/1500 [00:00<?, ? examples/s]

/home/worachotn/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3848: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Running tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/150 [00:00<?, ? examples/s]

/home/worachotn/.local/lib/python3.10/site-packages/accelerate/accelerator.py:527: FutureWarning: The `use_fp16` property is deprecated and will be removed in version 1.0 of Accelerate use `Accelerator.mixed_precision == 'fp16'` instead.
  warnings.warn(


In [22]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'negative_input_ids', 'labels', 'negative_labels'],
    num_rows: 1500
})

In [23]:
for step, batch in enumerate(train_dataloader):
  # print(tokenizer.decode(batch['input_ids'][0]))
    print(tokenizer.decode(batch['decoder_input_ids'][0]))
    print(tokenizer.decode(batch['decoder_input_ids'][1]))
    print(tokenizer.decode(batch['decoder_input_ids'][2]))
    print(tokenizer.decode(batch['decoder_input_ids'][3]))
    break

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


</s><s>Topic of Summary: search for books. Summary: #Person2# is struggling to choose the books about Hollywood in the 30s and 40s. #Person1# suggests he narrow the topic down by listing the specific years he wants. #Person1# tells #Person2# he can look up magazine articles.</s><pad><pad><pad><pad><pad>
</s><s>Topic of Summary: English idioms. Summary: Ms. Parker introduces English idioms to #Person1#. #Person1# thinks it's interesting.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
</s><s>Topic of Summary: search for books. Summary: #Person2# is struggling to choose the books about Hollywood in the 30s and 40s. #Person1# suggests he narrow the topic down by listing the specific years he wants. #Person1# tells #Person2# he can look up magazine articles.</s><pad><pad><pad><pad><pad>
</s><s>Topic of Summary: English idioms. Summary: Ms. Parker intr

In [26]:
for step, batch in enumerate(test_dataloader):
    print(batch.keys())
    break

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])


In [27]:
for step, batch in enumerate(eval_dataloader):
    print(batch.keys())
    break

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])


In [24]:
# = = = Training Preparation = = =
# Optimizer
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]

if args.ctrlen_model:
    no_decay_emb_matrix = ["bias", "LayerNorm.weight", "shared"]
else:
    no_decay_emb_matrix = ["bias", "LayerNorm.weight"]

optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay_emb_matrix)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]

if args.ctrlen_model:
    if args.model_type == 'bart':
        optimizer_grouped_parameters.extend([{
            "params": model.seq2seq_model.model.shared.parameters(),
            "lr": args.embedding_lr}])
    elif args.model_type == 't5':
        optimizer_grouped_parameters.extend([{
            "params": model.seq2seq_model.shared.parameters(),
            "lr": args.embedding_lr}])
    else:
        raise ValueError('{} model type not implemented'.format(args.model_type))

# optimizer
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

/home/worachotn/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
model, optimizer, train_dataloader, eval_dataloader, test_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, test_dataloader
)

In [26]:
# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
else:
    args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.max_train_steps,
)

# = = = = = = = = = = = = = = = = Train = = = = = = = = = = = = = = = = = = =
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f" Num examples = {len(train_dataset)}")
logger.info(f" Num Epochs = {args.num_train_epochs}")
logger.info(f" Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(f" Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f" Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f" Total optimization steps = {args.max_train_steps}")

# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), desc="Training: ", disable=not accelerator.is_local_main_process)
completed_steps = 0

val_results = []
acc_losses  = []
best_r2_f1  = None
best_epoch  = 0

if args.model_type == 'bart' or args.model_type == 't5':
    task_specific_params = model.config.task_specific_params
    params = task_specific_params.get('summarization', {})
    params['min_length'] = args.min_target_length
    params['max_length'] = args.max_target_length
    params['length_penalty'] = args.length_penalty
    params['num_beams'] = args.num_beams
    model.config.update(params)
else:
    raise ValueError('{} model type not implemented'.format(args.model_type))

10/28/2023 13:08:59 - INFO - __main__ - ***** Running training *****
10/28/2023 13:08:59 - INFO - __main__ -  Num examples = 1500
10/28/2023 13:08:59 - INFO - __main__ -  Num Epochs = 2
10/28/2023 13:08:59 - INFO - __main__ -  Instantaneous batch size per device = 2
10/28/2023 13:08:59 - INFO - __main__ -  Total train batch size (w. parallel, distributed & accumulation) = 128
10/28/2023 13:08:59 - INFO - __main__ -  Gradient Accumulation steps = 64
10/28/2023 13:08:59 - INFO - __main__ -  Total optimization steps = 24


Training:   0%|          | 0/24 [00:00<?, ?it/s]

In [27]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [28]:
for step, batch in enumerate(train_dataloader):
    print(batch['input_ids'].shape[1])
    break

256


In [29]:
# = = = = = = = = = = = = = = = = Train = = = = = = = = = = = = = = = = = = =
total_batch_size = args.per_device_train_batch_size * \
    accelerator.num_processes * args.gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f" Num examples = {len(train_dataset)}")
logger.info(f" Num Epochs = {args.num_train_epochs}")
logger.info(
    f" Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(
    f" Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(
    f" Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f" Total optimization steps = {args.max_train_steps}")

# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), desc="Training: ",
                    disable=not accelerator.is_local_main_process)
completed_steps = 0

val_results = []
acc_losses = []
best_r2_f1 = None
best_epoch = 0

# edit #
if args.model_type == 'bart' or args.model_type == 't5':
    # task_specific_params = model.module.config.task_specific_params
    task_specific_params = model.config.task_specific_params
    params = task_specific_params.get('summarization', {})
    params['min_length'] = args.min_target_length
    params['max_length'] = args.max_target_length
    params['length_penalty'] = args.length_penalty
    params['num_beams'] = args.num_beams
    # model.module.config.update(params)
    model.config.update(params)
else:
    raise ValueError(
        '{} model type not implemented'.format(args.model_type))

loss_list = []
train_loss_list = []
val_loss_list = []
last_output = None
hidden_states = None

# =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  = Train =  =  =  =  =  =  =  =  =  =  =  =  =  =  =
for epoch in range(args.num_train_epochs):
    # train
    model.train()
    epoch_loss = 0.0
    for step, batch in enumerate(train_dataloader):

        if args.ctrlen_model:  # CTRLen model
            outputs, loss = model(batch, tokenizer)
        # w/ and w/o label smoothing (always better with label smoothing)
        else:
            if args.label_smoothing == 0:
                outputs = model(**batch)
                loss = outputs.loss
            else:
                outputs = model(**batch, output_hidden_states=True)
                output_logits = outputs.logits

                output_probs = torch.nn.functional.log_softmax(
                    output_logits, dim=-1)
                
                if args.contrastive_loss:

                    max_encoder_token = batch['input_ids'].shape[1]
                    # max_encoder_token = model.config.max_position_embeddings

                    divide_num = int(output_probs.shape[0] / 2)
                    
                    # positive_embeddings_1 = outputs.encoder_last_hidden_state[0]
                    # positive_embeddings_2 = outputs.encoder_last_hidden_state[1]
                    # negative_embeddings_1 = outputs.encoder_last_hidden_state[2]
                    # negative_embeddings_2 = outputs.encoder_last_hidden_state[3]

                    # positive_decoder_embeddings_1 = outputs.decoder_hidden_states[-1][0]
                    # positive_decoder_embeddings_2 = outputs.decoder_hidden_states[-1][1]
                    # negative_decoder_embeddings_1 = outputs.decoder_hidden_states[-1][2]
                    # negative_decoder_embeddings_2 = outputs.decoder_hidden_states[-1][3]

                    # loss_1 = cosine_loss(positive_embeddings_1, negative_embeddings_1, contrastive)
                    # loss_2 = cosine_loss(positive_embeddings_2, negative_embeddings_2, contrastive)
                    # loss_1_decoder = cosine_loss(positive_decoder_embeddings_1, negative_decoder_embeddings_1, -1 * torch.ones(positive_decoder_embeddings_1.size(dim=0)).to(device))
                    # loss_2_decoder = cosine_loss(positive_decoder_embeddings_2, negative_decoder_embeddings_2, -1 * torch.ones(positive_decoder_embeddings_2.size(dim=0)).to(device))
                    # loss_cs = (loss_1 + loss_2) / 2
                    # loss_cs_decoder = (loss_1_decoder + loss_2_decoder) / 2
                    
                    positive_embeddings = outputs.encoder_last_hidden_state[:divide_num,:,:max_encoder_token]
                    negative_embeddings = outputs.encoder_last_hidden_state[divide_num:,:,:max_encoder_token]
                    positive_embeddings = positive_embeddings.view(-1, max_encoder_token)
                    negative_embeddings = negative_embeddings.view(-1, max_encoder_token)
                    contrastive = -1 * torch.ones(positive_embeddings.size(dim=0)).to(device)

                    model.config.max_position_embeddings

                    loss_cs = cosine_embedding_loss(positive_embeddings, negative_embeddings, contrastive)
                    
                    output_probs_pos = output_probs[:divide_num,:,:]
                    output_probs_pos = output_probs_pos.view(-1,
                                                     model.config.vocab_size)
                    output_probs_neg = output_probs[divide_num:,:,:]
                    output_probs_neg = output_probs_neg.view(-1,
                                                     model.config.vocab_size)
                    
    
                    gt_logits = batch['labels'][:divide_num]
                    gt_logits = gt_logits.view(-1)

                    loss_nll, nll = label_smoothed_nll_loss(
                        output_probs_pos, gt_logits, args.label_smoothing, ignore_index=tokenizer.pad_token_id)
                    
                    # (pos, neg, target, ignore_index=-100, ,device)
                    target_one = torch.ones(gt_logits.shape[0]).to(device)
                    loss_mr = margin_ranking_loss(output_probs_pos, output_probs_neg, 
                                                              gt_logits, target_one, ignore_index=tokenizer.pad_token_id)
                    # print(f"margin_ranking_loss: {loss_margin_ranking}")
                    loss = loss_nll + (args.alpha * loss_cs) + (args.alpha * loss_mr)
                    # loss = loss_nll + (args.alpha * loss_cs)
                    print(loss)
                    break
                    
                else:
                    output_probs = output_probs
                    output_probs = output_probs.view(-1,
                                                     model.config.vocab_size)
                    
    
                    gt_logits = batch['labels']
                    gt_logits = gt_logits.view(-1)
                
                    loss_nll, nll = label_smoothed_nll_loss(
                        output_probs, gt_logits, args.label_smoothing, ignore_index=tokenizer.pad_token_id)
    
                    loss = loss_nll           
                
        break
        acc_losses.append(loss.item())
        loss_list.append(loss)
        epoch_loss += loss.item()
        loss = loss / args.gradient_accumulation_steps
        # print(f"loss_grad: {loss}")
        accelerator.backward(loss)
        # break

        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            progress_bar.set_postfix(lr=lr_scheduler.get_last_lr()[
                                     0], loss=np.mean(acc_losses[-50:]))
            completed_steps += 1
            train_loss_list.append(epoch_loss/len(batch))

        if completed_steps >= args.max_train_steps:
            break

10/28/2023 13:08:59 - INFO - __main__ - ***** Running training *****
10/28/2023 13:08:59 - INFO - __main__ -  Num examples = 1500
10/28/2023 13:08:59 - INFO - __main__ -  Num Epochs = 2
10/28/2023 13:08:59 - INFO - __main__ -  Instantaneous batch size per device = 2
10/28/2023 13:08:59 - INFO - __main__ -  Total train batch size (w. parallel, distributed & accumulation) = 128
10/28/2023 13:08:59 - INFO - __main__ -  Gradient Accumulation steps = 64
10/28/2023 13:08:59 - INFO - __main__ -  Total optimization steps = 24


Training:   0%|          | 0/24 [00:00<?, ?it/s]

RuntimeError: shape '[-1, 1024]' is invalid for input of size 67712

In [31]:
outputs.encoder_last_hidden_state[:2,:,:].shape

torch.Size([2, 264, 1024])

In [32]:
outputs.encoder_last_hidden_state[2:,:,:].shape

torch.Size([2, 264, 1024])

In [ ]:
output_probs.shape

In [158]:
int(output_probs.shape[0] / 2)

2

In [145]:
output_probs_pos = output_probs[:2,:,:]
output_probs_pos = output_probs_pos.view(-1,
                                 model.config.vocab_size)
output_probs_neg = output_probs[2:,:,:]
output_probs_neg = output_probs_neg.view(-1,
                                 model.config.vocab_size)
gt_logits = batch['labels'][:2]
gt_logits = gt_logits.view(-1)

In [146]:
output_probs_pos.shape

torch.Size([80, 50265])

In [147]:
output_probs_neg.shape

torch.Size([80, 50265])

In [39]:
gt_logits.shape

torch.Size([96])

In [149]:
probs_pos = output_probs_pos[~gt_logits.eq(-100)]
probs_neg = output_probs_neg[~gt_logits.eq(-100)]

In [150]:
gt = gt_logits[~gt_logits.eq(-100)]

In [151]:
if gt.dim() == probs_pos.dim() - 1:
    gt = gt.unsqueeze(-1)

In [152]:
nll_pos = -probs_pos.gather(dim=-1, index=gt)
nll_neg = -probs_neg.gather(dim=-1, index=gt)

In [153]:
nll_sq_pos = nll_pos.squeeze(-1)
nll_sq_neg = nll_neg.squeeze(-1)

In [154]:
target_one = torch.ones(nll_sq_pos.shape[0])

In [155]:
import torch.nn as nn
loss = nn.MarginRankingLoss()
input1 = nll_sq_pos
input2 = nll_sq_neg
target = target_one.to("cuda:0")
output = loss(input1, input2, target)
# output.backward()

In [156]:
output

tensor(3.3658, device='cuda:0', grad_fn=<MeanBackward0>)

In [ ]:
len(outputs.encoder_last_hidden_state)

2

In [ ]:
outputs.encoder_last_hidden_state

tensor([[[ 0.0199,  0.0383,  0.0413,  ..., -0.0192,  0.0033, -0.0034],
         [-0.1682, -0.0113,  0.1301,  ..., -0.0883, -0.0093, -0.1615],
         [-0.0111, -0.4472,  0.0442,  ...,  0.0226, -0.0170, -0.2872],
         ...,
         [-0.0308, -0.2955, -0.3913,  ...,  0.0104, -0.0009, -0.0810],
         [-0.0451, -0.4547, -0.5451,  ...,  0.1203, -0.0126,  0.1627],
         [-0.0310, -0.0535, -0.3161,  ...,  0.0269, -0.0641,  0.1126]],

        [[ 0.0211,  0.0319,  0.0484,  ..., -0.0260, -0.0024, -0.0104],
         [-0.2227, -0.0779, -0.0718,  ...,  0.0067, -0.0177, -0.0376],
         [-0.1187, -0.5692, -0.0248,  ..., -0.0780,  0.0284, -0.4738],
         ...,
         [-0.0440,  0.1978, -0.3247,  ..., -0.0017,  0.1312,  0.3230],
         [-0.1271, -0.3494, -0.5145,  ...,  0.0051,  0.1007,  0.0965],
         [-0.1246, -0.2684, -0.6091,  ...,  0.0990,  0.0489, -0.0175]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)

In [ ]:
outputs.encoder_last_hidden_state.size(dim=1)

296

In [ ]:
print(len(outputs.logits))
print(outputs.logits[0].size())

2
torch.Size([48, 50266])


In [ ]:
cosine_loss = torch.nn.CosineEmbeddingLoss()

In [ ]:
loss_cs = cosine_loss(outputs.encoder_last_hidden_state[0], outputs.encoder_last_hidden_state[1], torch.ones(outputs.encoder_last_hidden_state.size(dim=1)).to(torch.device('cuda')))

In [ ]:
loss_cs

tensor(0.7458, device='cuda:0', grad_fn=<MeanBackward0>)

In [ ]:
# =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  = Test =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =
# load best model
logger.info("Loading Best Result is at epoch {} for Testing".format(best_epoch))

unwrapped_model = accelerator.unwrap_model(model)
config          = config.from_pretrained(args.output_dir+'/best')
tokenizer       = tokenizer.from_pretrained(args.output_dir+'/best', config=config)
unwrapped_model = unwrapped_model.from_pretrained(args.output_dir+'/best', config=config)
model           = accelerator.prepare(unwrapped_model)

if args.model_type == 'bart' or args.model_type == 't5':
    task_specific_params = model.config.task_specific_params
    params = task_specific_params.get('summarization', {})
    params['min_length'] = args.min_target_length
    params['max_length'] = args.max_target_length
    params['length_penalty'] = args.length_penalty
    params['num_beams'] = args.num_beams
    model.config.update(params)
else:
    raise ValueError('{} model type not implemented'.format(args.model_type))

# start Test
logger.info("Collecting Testing Result...")
model.eval()

test_predict     = []
test_groundtruth = []
for step, batch in enumerate(tqdm(test_dataloader, leave=False)):
    with torch.no_grad():
        generated_tokens = accelerator.unwrap_model(model).generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
        )

        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = batch["labels"]

        if not args.pad_to_max_length:
            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(batch["labels"], dim=1, pad_index=tokenizer.pad_token_id)

        generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
        labels = accelerator.gather(labels).cpu().numpy()

        if args.ignore_pad_token_for_loss:
            # Replace -100 in the labels as we can't decode them.
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        if isinstance(generated_tokens, tuple):
            generated_tokens = generated_tokens[0]

        decoded_preds  = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        decoded_preds  = [' '.join(sent.split('\n')) for sent in decoded_preds]
        decoded_labels = [' '.join(sent.split('\n')) for sent in decoded_labels]

        test_predict.extend(decoded_preds)
        test_groundtruth.extend(decoded_labels)

print(raw_datasets['test']['dialogue'][0])

if args.len_output == 'real':
    new_test_predict = []
    for sample in test_predict:
        try:
            gen_sum = sample.split('Summary: ')[2]
            new_test_predict.append(gen_sum)
        except:
            new_test_predict.append(sample)
    test_predict = new_test_predict

logger.info("")
logger.info("ROUGE score on test set")
test_scores = py_rouge_scores(test_predict, test_groundtruth)
logger.info("")


# Save generated summaries
if args.len_input == 'predict':
    os.makedirs(args.output_dir+'/predict_gen_samples', exist_ok=True)
else:
    os.makedirs(args.output_dir+'/gen_samples', exist_ok=True)

for i in range(len(test_predict)):
    test_id        = raw_datasets['test']['id'][i]
    test_dialogue  = raw_datasets['test']['dialogue'][i]
    test_summary   = raw_datasets['test']['summary'][i]
    test_predict_s = test_predict[i]

    if args.len_input == 'predict':
        with open(args.output_dir+'/predict_gen_samples/'+str(test_id)+'.txt', 'w') as f:
            test_dialogue = test_dialogue.encode('ascii', 'ignore').decode('ascii')
            f.write(test_dialogue)
            f.write('\n\n')
            f.write('Golden Summary:\n')
            test_summary = test_summary.encode('ascii', 'ignore').decode('ascii')
            f.write(test_summary)
            f.write('\n\n')
            f.write('Generate Summary:\n')
            test_predict_s = test_predict_s.encode('ascii', 'ignore').decode('ascii')
            f.write(test_predict_s)
    else:
        with open(args.output_dir+'/gen_samples/'+str(test_id)+'.txt', 'w') as f:
            test_dialogue = test_dialogue.encode('ascii', 'ignore').decode('ascii')
            f.write(test_dialogue)
            f.write('\n\n')
            f.write('Golden Summary:\n')
            test_summary = test_summary.encode('ascii', 'ignore').decode('ascii')
            f.write(test_summary)
            f.write('\n\n')
            f.write('Generate Summary:\n')
            test_predict_s = test_predict_s.encode('ascii', 'ignore').decode('ascii')
            f.write(test_predict_s)

10/04/2023 19:07:23 - INFO - __main__ - Loading Best Result is at epoch 4 for Testing
loading configuration file ./output/1-bart-baseline-loss/best/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"

  0%|          | 0/19 [00:00<?, ?it/s]

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "max_length": 128,
  "min_length": 1,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.33.3"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "max_length": 128,
  "min_length": 1,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.33.3"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "max_length": 128,
  "min_length": 1,
  "no_re

Topic of Summary: communication method. Length of Summary: 27. Dialogue: # Person1 # : Ms. Dawson , I need you to take a dictation for me . # Person2 # : Yes , sir ... # Person1 # : This should go out as an intra-office memorandum to all employees by this afternoon . Are you ready ? # Person2 # : Yes , sir . Go ahead . # Person1 # : Attention all staff ... Effective immediately , all office [TAG]communications[TAG] are restricted to email correspondence and official memos . The use of Instant Message programs by employees during working hours is strictly prohibited . # Person2 # : Sir , does this apply to intra-office [TAG]communications[TAG] only ? Or will it also restrict external [TAG]communications[TAG] ? # Person1 # : It should apply to all [TAG]communications[TAG] , not only in this office between employees , but also any outside [TAG]communications[TAG] . # Person2 # : But sir , many employees use Instant Messaging to communicate with their clients . # Person1 # : They will just

10/04/2023 19:08:05 - INFO - root - 
10/04/2023 19:08:05 - INFO - root - 	rouge-1:	P: 37.12	R: 47.88	F1: 40.73
10/04/2023 19:08:05 - INFO - root - 	rouge-2:	P: 13.30	R: 17.25	F1: 14.64
10/04/2023 19:08:05 - INFO - root - 	rouge-l:	P: 35.01	R: 43.23	F1: 37.96
10/04/2023 19:08:05 - INFO - root - 
10/04/2023 19:08:05 - INFO - __main__ - 
